In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from scipy.stats import chi2
import researchpy

In [2]:
path = r'C:\Users\User\Desktop\University of Manchester\MSc Data Science\Semester 2\Dissertation\Dataset\IHDS data\Linking file'
link = pd.read_csv(path + '/' + 'linkind_1.csv')

In [3]:
# Compute wave 1 (2005) IDHH
# Extract the number from stateID string
def extract_number(input_string):
    x = np.array(input_string.split())
    res = x[np.char.isnumeric(x)].astype(int)
    return res[0]
link['STATEID'] = link['STATEID'].apply(extract_number)

In [4]:
# Drop rows with missing values in PERSONID2005
link.dropna(inplace = True)
# leaving only 150988 reinterviewed individuals in the linking file dataset

In [5]:
# Drop irrelevant columns
link = link.drop(columns=['HHWAVES', 'PWAVES'])

In [6]:
# Convert the dtype of all columns to float to avoid problem of overflowing in int64
link = link.astype(float)

In [7]:
# Create CASEID_2005 in linking file
link['CASEID_2005'] = link['STATEID']*(10**9) + link['DISTID']*(10**7) + link['PSUID']*(10**5) + link['HHID2005']*(10**3) + link['HHSPLITID2005']*(10**2) + link['PERSONID2005']

# impute missing values in CASEID_2005 with 9999999999
#link['CASEID_2005'].fillna(9999999999, inplace = True)


In [8]:
# Create CASEID_2012 in linking file
# There is a typo in HHID, every digit is mistakenly multiplied by 10, we handle it by changing the CASEID_2012 formula
link['CASEID_2012'] = link['STATEID']*(10**10) + link['DISTID']*(10**8) + link['PSUID']*(10**6) + link['HHID']*(10**3) + link['HHSPLITID']*(10**2) + link['PERSONID']

In [9]:
# Select only the columns required for matching waves 1 and 2 data
link = link[['CASEID_2005', 'CASEID_2012']]

In [10]:
# Save linking file as csv file
link.to_csv(path + '/' + 'linking file_clean.csv', index = False)

In [16]:
# Merge linking files with IHDS wave 1 data
path = r'C:\Users\User\Desktop\University of Manchester\MSc Data Science\Semester 2\Dissertation\Dataset\IHDS data\IHDS_2005\IHDS_2005 csv file'
df = pd.read_csv(path + '/' + 'df_child_wave1.csv')

# rename the parental occup columns
df = df.rename(columns={'paternal_occup': 'paternal_occup_w1', 'maternal_occup': 'maternal_occup_w1', 'CASEID':'CASEID_2005'})

# select the required columns
df = df[['CASEID_2005', 'paternal_occup_w1', 'maternal_occup_w1']]

# save as csv file
df.to_csv(path + '/' + 'df_child_wave1_for merging.csv', index = False)


C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
# Use linking file to merge w1 and w2 datasets
path = r'C:\Users\User\Desktop\University of Manchester\MSc Data Science\Semester 2\Dissertation\Dataset\IHDS data\IHDS_2005\IHDS_2005 csv file'
w1 = pd.read_csv(path + '/' + 'df_child_wave1_for merging.csv')

path2 = r'C:\Users\User\Desktop\University of Manchester\MSc Data Science\Semester 2\Dissertation\Dataset\IHDS data\IHDS_2012\IHDS_2012 csv file'
w2 = pd.read_csv(path2 + '/' + 'IHDS_2012_cleaned_df_child_2023_07_10_v2.csv')

path3 = r'C:\Users\User\Desktop\University of Manchester\MSc Data Science\Semester 2\Dissertation\Dataset\IHDS data\Linking file'
link = pd.read_csv(path3 + '/' + 'linking file_clean.csv')

In [17]:
# rename IDPERSON in IHDS2012 to CASEID_2012
w2 = w2.rename(columns = {'IDPERSON':'CASEID_2012'})

In [20]:
w1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62580 entries, 0 to 62579
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CASEID_2005        62580 non-null  int64 
 1   paternal_occup_w1  55961 non-null  object
 2   maternal_occup_w1  59635 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [21]:
link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150988 entries, 0 to 150987
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   CASEID_2005  150988 non-null  float64
 1   CASEID_2012  150988 non-null  float64
dtypes: float64(2)
memory usage: 2.3 MB


In [22]:
# Merge w1 and linking file
merge_df = w1.merge(link, on = 'CASEID_2005', how = 'left')

In [23]:
# Drop rows from merge_df in which contain missing values in CASEID_2012
merge_df.dropna(subset=['CASEID_2012'], inplace = True)

In [24]:
merge_df.to_csv(path + '/'+ 'w1_link.csv')

In [25]:
merge_df2 = w2.merge(merge_df, on = 'CASEID_2012', how='left')

In [26]:
merge_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51533 entries, 0 to 51532
Data columns (total 85 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Unnamed: 0                                              51533 non-null  int64  
 1   SURVEY_x                                                51533 non-null  object 
 2   STATEID_x                                               51533 non-null  object 
 3   DISTID_x                                                51533 non-null  int64  
 4   PSUID_x                                                 51533 non-null  int64  
 5   HHID_x                                                  51533 non-null  int64  
 6   HHSPLITID_x                                             51533 non-null  int64  
 7   PERSONID                                                51533 non-null  float64
 8   IDPSU_x                             

In [27]:
merge_df2.to_csv(path + '/'+ 'w1_w2_merged.csv')

# Run Cramer's V test on merged dataset

In [28]:
df = pd.read_csv(path + '/'+ 'w1_w2_merged.csv')

In [29]:
# convert Cl_incidence to binary variable
df['CL_incidence'] = df['CL_incidence'].replace([0,1],['No','Yes'])

In [30]:
# create df2 by selecting 2 columns
df2 = df[['paternal_occup_w1', 'CL_incidence']]

In [31]:
df2['paternal_occup_w1'].isna().sum()

30906

In [32]:
# remove rows with missing values in paternal_occup_w1
df2.dropna(axis = 0, inplace= True)
# 30906 rows are dropped

C:\Users\User\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [33]:
myfield1 = df2['paternal_occup_w1']
myfield2 = df2['CL_incidence']

In [34]:
contTable = pd.crosstab(myfield1, myfield2)

In [35]:
contTable

CL_incidence,No,Yes
paternal_occup_w1,,
"Construction, Production & Labour",4819,952
"Farmers, Fishermen & Hunters",3611,995
Others,6290,826
Professional & Managerial,617,26
Sales workers,354,23
Service & Clerical,1947,167


In [37]:
# Run the Cramer's V test with researchpy package
crosstab, res = researchpy.crosstab(df2['paternal_occup_w1'], df2['CL_incidence'], test = 'chi-square')

In [38]:
res

,Chi-square test,results
0,Pearson Chi-square ( 5.0) =,406.6342
1,p-value =,0.0000
2,Cramer's V =,0.1404


In [39]:
# Repeat for maternal occup w1
df3 = df[['maternal_occup_w1', 'CL_incidence']]
df3.dropna(axis = 0, inplace= True)

# Run the Cramer's V test with researchpy package
crosstab, res = researchpy.crosstab(df3['maternal_occup_w1'], df3['CL_incidence'], test = 'chi-square')

C:\Users\User\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [41]:
contTable = pd.crosstab(df3['maternal_occup_w1'], df3['CL_incidence'])

In [42]:
contTable

CL_incidence,No,Yes
maternal_occup_w1,,
"Construction, Production & Labour",918,277
"Farmers, Fishermen & Hunters",2756,893
Others,14472,1935
Professional & Managerial,178,5
Sales workers,20,1
Service & Clerical,316,42


In [40]:
res

,Chi-square test,results
0,Pearson Chi-square ( 5.0) =,487.7508
1,p-value =,0.0000
2,Cramer's V =,0.1495


In [ ]:
# PLotting diagram for CL incidnece and parental occupations
